In [1]:
from together import Together
import pandas as pd
import numpy as np
from datasets import load_dataset
from tokens import tokens
import os
import dspy
import sys
sys.path.insert(0, r"C:\Users\mikad\Documents\GitHub\textgrad")
import textgrad
from textgrad.engine import get_engine
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss
from textgrad.tasks import DataLoader 
import concurrent
from tqdm import tqdm
import random
import math

c:\Users\mikad\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\mikad\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## Initialization

In [2]:
# set up tokens
os.environ['TOGETHER_API_KEY'] = together_token
os.environ['HF_TOKEN'] = hf_token

In [4]:
# Load testing data
dataset = load_dataset("gtfintechlab/fomc_communication")
sample = dataset['test'][0]
sentence = sample['sentence']
label = sample['label']
print(f"Original sentence: {sentence}\nLabel: {label}")

Original sentence: Participants agreed that the labor market had remained strong over the intermeeting period and that economic activity had risen at a moderate rate.
Label: 2


In [16]:
#finqa (qa), ECTSum (summarization) 
dataset = load_dataset("gtfintechlab/finqa")
sample = dataset['test'][0]

## Textgrad Prompt Optimization

In [4]:
def find_first_occurrence(text):
    words = ["hawkish", "dovish", "neutral"]
    positions = [(text.find(word), word) for word in words if text.find(word) != -1]
    return min(positions)[1] if positions else 'neutral'

def extract_answer(text):
    client = Together()
    try: 
        model_response = client.chat.completions.create(
            model='meta-llama/Meta-Llama-3-8b-Instruct',
            messages=[{'role': 'system', 'content': "Discard all previous instructions. Behave as if you are an text extraction model. You are only extracting the answer."},
                        {'role': 'user', 'content': f"Extract the one word answer (HAWKISH, DOVISH, or NEUTRAL) from the following text: {text}.\nDo not enter any other text."}],
            max_tokens=128, temperature=0.7, top_k=50, top_p=0.7,repetition_penalty=1.1
        )
        response = model_response.choices[0].message.content
        return response.strip().split()[0]
    except:
        return None

def string_based_equality_fn(prediction: Variable, ground_truth_answer: Variable):
    mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
    # pred = find_first_occurrence(str(prediction.value).lower())
    pred = extract_answer(str(prediction.value).lower())
    if (pred != None and str(pred).lower() == str(mapping[int(ground_truth_answer.value)]).lower()):
        return 1
    else: 
        return 0

eval_fn = textgrad.autograd.StringBasedFunction(string_based_equality_fn, "Evaluate if LLM answer matches ground truth")

In [5]:
# Evaluate given sample using custom evaluation function
def eval_sample(x, y, eval_fn, model):
    x = Variable(x, requires_grad=False, role_description="query to the language model")
    y = Variable(y, requires_grad=False, role_description="correct answer for the query")
    response = model(x)
    inputs = dict(prediction = response, ground_truth_answer = y)
    eval_output_variable = eval_fn(inputs = inputs)
    return eval_output_variable
    
# Evaluate dataset using custom evaluation function
def eval_dataset(test_set, eval_fn, model, max_samples = None, max_workers = 24):
    if max_samples is None: 
        max_samples = len(test_set)
    accuracy_list = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for _, sample in enumerate(test_set):
            future = executor.submit(eval_sample, sample[0], sample[1], eval_fn, model)
            futures.append(future)
            if len(futures) >= max_samples:
                break
        tqdm_loader = tqdm(concurrent.futures.as_completed(futures), total=len(futures), position=0)
        for future in tqdm_loader:
            output = future.result()
            output = int(output.value)
            accuracy_list.append(output)
            tqdm_loader.set_description(f"Accuracy: {np.mean(accuracy_list)}")
    return accuracy_list 

# Evaluate system prompt and revert to previous prompt if new prompt has lower test accuracy
def run_val_revert(system_prompt, results, eval_fn, model, validation_dataset):
    curr_accuracy = np.mean(eval_dataset(validation_dataset, eval_fn, model))
    prev_accuracy = np.mean(results["val_accuracy"][-1])
    print(f"Val accuracy: {curr_accuracy}\nPrevious val accuracy: {prev_accuracy}")
    previous_prompt = results["prompt"][-1]

    results["new_prompt_val_accuracy"].append(curr_accuracy)
    
    if curr_accuracy < prev_accuracy:
        print(f"Rejected prompt: {system_prompt.value}")
        system_prompt.set_value(previous_prompt)
        curr_accuracy = prev_accuracy

    results["val_accuracy"].append(curr_accuracy)

In [6]:
engine = get_engine('together-meta-llama/Meta-Llama-3-8b-Instruct-Turbo')
# production use 70b model

starting_prompt = "Classify the following sentence from FOMC into hawkish, dovish, or neutral based on its stance on monetary policy. Explain your reasoning."
starting_prompt = "Behave like you are an expert at summarization tasks. Below an earnings call transcript of a Russell 3000 Index company is provided. Perform extractive summarization followed by paraphrasing the transcript in bullet point format according to the experts-written short telegram-style bullet point summaries derived from corresponding Reuters articles. The target length of the summary should be at most 50 words."
system_prompt = Variable(starting_prompt, requires_grad=True, role_description="System prompt to the language model. Must include instructions to output one of hawkish, dovish, and neutral.")
model = textgrad.BlackboxLLM(engine, system_prompt, max_tokens=64, temperature=0.7, top_k=50, top_p=0.7, repetition_penalty=1.1)

optimizer = TextualGradientDescent(engine=engine, parameters=[system_prompt], constraints = ["Must include text instructions to output one of hawkish, dovish, and neutral, written with those exact labels and not synonyms."],
                max_tokens=150, temperature=0.9, top_k=50, top_p=0.7, repetition_penalty=1.1)

In [7]:
# testing zero-shot performance
mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
data = [(data['sentence'], data['label']) for data in dataset['test']]
results = {"test_accuracy": [], "prompt": []}
results["test_accuracy"].append(eval_dataset(data, eval_fn, model))
results["prompt"].append(system_prompt.get_value())

Accuracy: 0.4012096774193548: 100%|██████████| 496/496 [00:00<00:00, 549.01it/s] 


In [7]:
# subset the data to do a mini run
def pick_random(data, n = 20):
    labels = {0: [], 1: [], 2: []}
    for sentence, label in data:
        labels[label].append((sentence, label))
    result = []
    for label in labels:
        result.extend(random.sample(labels[label], n))
    return result
training = [(data['sentence'], data['label']) for data in dataset['train']]
random.shuffle(training)
training_data_options = training[:math.ceil(len(training)*0.8)]
val_data_options = training[math.ceil(len(training)*0.8):]

num_classes = 3
training_count = math.ceil(len(training) * 0.5)
training_data = pick_random(training_data_options, n = math.ceil(training_count // num_classes))
val_count = math.ceil(len(training) * 0.1)
val_data = pick_random(val_data_options, n = math.ceil(val_count // num_classes))
testing_data = [(data['sentence'], data['label']) for data in dataset['test']]

In [11]:
training = [(data['context'], data['response']) for data in dataset['train']]
random.shuffle(training)
training_data_options = training[:math.ceil(len(training)*0.8)]
val_data_options = training[math.ceil(len(training)*0.8):]

training_count = math.ceil(len(training) * 0.5)
training_data = random.sample(training_data_options, training_count)
val_count = math.ceil(len(training) * 0.1)
val_data = random.sample(val_data_options, val_count)
testing_data = [(data['context'], data['response']) for data in dataset['test']]

In [8]:
start_val_accuracy = np.mean(eval_dataset(val_data, eval_fn, model))
results = {"val_accuracy": [start_val_accuracy], "prompt": [starting_prompt], 
            "new_prompt": [starting_prompt], "new_prompt_val_accuracy": [start_val_accuracy]}
print(f"Starting prompt: {starting_prompt}")
print(f"Starting val accuracy: {start_val_accuracy}")

# optimize system prompt
train_loader = DataLoader(training_data, batch_size=5, shuffle=True)

for epoch in range(20):
    for step, (batch_x, batch_y) in enumerate((pbar := tqdm(train_loader, position=0))):
        # initialize training step
        if step >= 10:
            break
        pbar.set_description(f"Epoch {epoch}: Training step {step}")
        optimizer.zero_grad()
        losses = []

        # train off items in batch
        for (x, y) in zip(batch_x, batch_y):
            eval_output = eval_sample(x, y, eval_fn, model)
            losses.append(eval_output)

        # sum losses
        try:
            total_loss = textgrad.sum(losses)
            total_loss.backward(engine)
            optimizer.step()
        except:
            continue

        print("Prompt: ", system_prompt.get_value())

    # evaluate new system prompt
    # if worse, revert to previous prompt
    results["new_prompt"].append(system_prompt.get_value())
    print("New prompt: ", system_prompt.get_value())
    run_val_revert(system_prompt, results, eval_fn, model, val_data)
    
    print("System prompt: ", system_prompt)
    results["prompt"].append(system_prompt.get_value())

Accuracy: 0.4090909090909091: 100%|██████████| 198/198 [00:03<00:00, 62.40it/s]  


Starting prompt: Classify the following sentence from FOMC into hawkish, dovish, or neutral based on its stance on monetary policy. Output your prediction on the first line and your explanation after.
Starting val accuracy: 0.4090909090909091


Epoch 0: Training step 1: : 1it [00:19, 19.30s/it]

Prompt:  Classify the following sentence from FOMC into hawkish, dovish, or neutral, considering the tone, syntax, and semantics of the sentence, as well as the presence or absence of urgency or concern. Provide a detailed explanation for your prediction, including specific examples from the sentence that support your classification. What are the key factors driving your classification, and how do they relate to the broader context of monetary policy? Output your prediction on the first line and your explanation after, including specific reasons and potential drawbacks. Additionally, acknowledge the complexity of the topic and the limitations of your model. Please provide a brief summary of the sentence being classified to help the reader understand the context of your classification. Your explanation should be clear and


Epoch 0: Training step 2: : 2it [00:39, 19.84s/it]

...

Prompt:  VARIABLE> Classify the following sentence from FOMC into one of hawkish, dovish, or neutral, providing a precise and specific classification, along with a clear and concise justification for your answer. Specifically, please:

* Provide a detailed analysis of the sentence's tone, sentiment, and language use, highlighting the key phrases, sentence structures, and tone markers that contribute to the sentence's overall tone and sentiment, while acknowledging potential biases and uncertainties in your analysis.
* Offer alternative classifications and explain why they are less likely or more plausible, considering the potential complexities and ambiguities of the input text, and explicitly addressing potential limitations and uncertainties in your model.
* Consider the following cues to inform your classification:
	+ The


Epoch 19: Training step 9: : 10it [03:28, 20.86s/it]


Prompt:  VARIABLE> Classify the following sentence from FOMC into one of hawkish, dovish, or neutral, providing a precise and specific classification, along with a clear and concise justification for your answer. Specifically, please:

* Provide a detailed analysis of the sentence's tone, sentiment, and language use, highlighting the key phrases, sentence structures, and tone markers that contribute to the sentence's overall tone and sentiment, while acknowledging potential biases and uncertainties in your analysis.
* Offer alternative classifications and explain why they are less likely or more plausible, considering the potential complexities and ambiguities of the input text, and explicitly addressing potential limitations and uncertainties in your model.
* Consider the following cues to inform your classification:
	+
New prompt:  VARIABLE> Classify the following sentence from FOMC into one of hawkish, dovish, or neutral, providing a precise and specific classification, along with a

Accuracy: 0.45454545454545453: 100%|██████████| 198/198 [00:07<00:00, 28.16it/s]

Val accuracy: 0.45454545454545453
Previous val accuracy: 0.5505050505050505
Rejected prompt: VARIABLE> Classify the following sentence from FOMC into one of hawkish, dovish, or neutral, providing a precise and specific classification, along with a clear and concise justification for your answer. Specifically, please:

* Provide a detailed analysis of the sentence's tone, sentiment, and language use, highlighting the key phrases, sentence structures, and tone markers that contribute to the sentence's overall tone and sentiment, while acknowledging potential biases and uncertainties in your analysis.
* Offer alternative classifications and explain why they are less likely or more plausible, considering the potential complexities and ambiguities of the input text, and explicitly addressing potential limitations and uncertainties in your model.
* Consider the following cues to inform your classification:
	+
System prompt:  VARIABLE> Classify the following sentence from FOMC into one of haw

In [9]:
results

{'val_accuracy': [0.4090909090909091,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505,
  0.5505050505050505],
 'prompt': ['Classify the following sentence from FOMC into hawkish, dovish, or neutral based on its stance on monetary policy. Output your prediction on the first line and your explanation after.',
  'VARIABLE> Classify the following sentence from FOMC into hawkish, dovish, or neutral, providing a detailed and nuanced explanation of the factors driving your classification. Specifically, highlight the key linguistic features, tone, and semantic cues that support your prediction, and acknowledge the potential limitat

In [9]:
pd.DataFrame(results).to_csv("textgrad_token_limited.csv", index=False)

In [12]:
engine = get_engine('together-mistralai/Mistral-7B-Instruct-v0.3')

starting_prompt = "Analyze the given FOMC statement and determine the expected federal funds rate or the pace of balance sheet reduction based on the statement. Interpret the statement in the context of the FOMC's overall monetary policy objectives and recent statements or actions. Provide a clear and detailed classification of the monetary policy stance, along with a justification for your decision. Choose from the following categories: Neutral, Dovish, Moderately Hawkish, or Very Hawkish. Explain the reasons for your prediction, focusing on the FOMC's monetary policy objectives and any indications of a shift in policy direction. Provide evidence or references that support your prediction, if possible. Discuss the potential impact on inflation, employment, and any associated risks or uncertainties, focusing on the short-term and medium-term implications. Discuss the potential risks and uncertainties associated with the predicted policy stance, and provide specific examples or scenarios that illustrate the short-term and medium-term implications of the FOMC's monetary policy stance on inflation, employment, and associated risks or uncertainties."
system_prompt = Variable(starting_prompt, requires_grad=True, role_description="system prompt to the language model")
model = textgrad.BlackboxLLM(engine, system_prompt)

optimizer = TextualGradientDescent(engine=engine, parameters=[system_prompt])

mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
data = [(data['sentence'], data['label']) for data in dataset['test']]
eval_dataset(data, eval_fn, model)

Accuracy: 0.5403225806451613: 100%|██████████| 496/496 [09:35<00:00,  1.16s/it]


[1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,


## Testing together 1.2 API

In [4]:
# the one in prompts.py was not returning just the system prompt/user message which together 1.2 needs
def fomc_prompt(sentence: str):

    system_prompt = f"""Discard all the previous instructions. Behave like you are an expert sentence clas-
                sifier."""
    user_msg = f"""Classify the following sentence from FOMC into ‘HAWKISH’, ‘DOVISH’, or ‘NEU-
                TRAL’ class. Label ‘HAWKISH’ if it is corresponding to tightening of the monetary policy,
                ‘DOVISH’ if it is corresponding to easing of the monetary policy, or ‘NEUTRAL’ if the
                stance is neutral. Provide the label in the first line and provide a short explanation in the
                second line. This is the sentence: {sentence}"""

    return system_prompt, user_msg

In [8]:
# Testing together 1.2 API
client = Together()
model_response = client.chat.completions.create(
    model='allenai/OLMo-7B-Instruct',
    messages=[{'role': 'system', 'content': fomc_prompt(sentence)[0]},
                {'role': 'user', 'content': fomc_prompt(sentence)[1]}],
    max_tokens=128,
    temperature=0.7,
    top_k=50,
    top_p=0.7,
    repetition_penalty=1.1
)
response_label = model_response.choices[0].message.content
token_usage = {'prompt_tokens': model_response.usage.prompt_tokens, 'response_tokens': model_response.usage.completion_tokens, 'total_tokens': model_response.usage.total_tokens}
print(f"LLM response: {response_label.strip()}\nToken usage: {token_usage}")

LLM response: Neutral class
Explanation: The sentence mentions that participants agreed that the labor market remained strong and economic activity rose at a moderate rate, indicating a neutral stance towards monetary policy.
Token usage: {'prompt_tokens': 174, 'response_tokens': 37, 'total_tokens': 211}


## Testing DSPy

In [54]:
# testing dspy
# dspy together module has some issues with how they wrote it (passing specific parameters is hard)
# rate limiting error messages
# TODO: look into dspy together module more to see how to fix it

lm = dspy.Together(model = 'mistralai/Mistral-7B-Instruct-v0.3', stop = ['</s>', '<s>'])
dspy.settings.configure(lm=lm)
d = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}

class StanceAnalysis(dspy.Signature):
    """Classify the sentence's stance on the monetary policy between hawkish, neutral, and dovish."""
    
    sentence = dspy.InputField()
    stance = dspy.OutputField(desc = "hawkish, neutral, or dovish")

class Analysis(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(StanceAnalysis)
    
    def forward(self, sentence):
        return self.predict(sentence=sentence)
    
analyze = Analysis()
analyze(sentence)

Backing off 0.6 seconds after 1 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 0.5 seconds after 2 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 3.4 seconds after 3 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 7.0 seconds after 4 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 12.9 seconds after 5

KeyboardInterrupt: 

## Initial textgrad testing

In [5]:
# basic textgrad together engine
engine = get_engine('together-allenai/OLMo-7B-Instruct')
engine(fomc_prompt(sentence)[1], system_prompt = fomc_prompt(sentence)[0], max_tokens=128, temperature=0.7, top_p=0.7)

'Neutral class\nExplanation: The sentence indicates that participants agreed on the strong labor market and moderate economic activity, suggesting a neutral stance towards monetary policy.'

In [9]:
system_prompt = 'Discard all the previous instructions. Behave like you are an expert sentence classifier.'
engine = get_engine('together-mistralai/Mistral-7B-Instruct-v0.3')

sys_prompt = Variable('Discard all the previous instructions. Behave like you are an expert sentence classifier.', role_description="The system prompt")
user_prompt = Variable("Classify the sentence's stance on the monetary policy between hawkish, neutral, and dovish.", role_description="The user prompt", requires_grad=True)
input_sentence = Variable(sentence, role_description="The input sentence")
loss = TextLoss(sys_prompt, engine=engine)

# optimization does not allow you to set any parameters besides prompt & system prompt
# meaning you can't set max tokens or temperature --> problem b/c default output tokens is 2000 
# so models with too small context windows will error

optimizer = TextualGradientDescent(parameters=[user_prompt], engine=engine)
l = loss(input_sentence)
l.backward(engine)
optimizer.step()

In [10]:
# optimized version of user prompt
user_prompt.value

'Determine the monetary policy stance (hawkish, neutral, or dovish) expressed in the given sentence.'